In [ ]:
import numpy
import os
import shutil
import random
import pandas
import subprocess
import shlex

SSH_USER = 'am72ghiassi'
DESIGN_CSV = 'experiments.csv' # The CSV with the experiment design
masterip = 'spark://10.132.0.4:7077'

In [ ]:
# Generate a random experiment ID
ex_id = ''.join(random.choice(string.ascii_lowercase)
                for i in range(8))
print(f"Experiment ID: {ex_id}")

experiments = pandas.read_csv(DESIGN_CSV)
experiments.columns.values[0] = 'Index'
experiments.set_index('Index')

os.makedirs(f"raw/{ex_id}", exist_ok=True)
shutil.copyfile(DESIGN_CSV, f"raw/{ex_id}/design.csv")

In [ ]:
for idx, row in experiments.iterrows():
    print(f"Experiment {row['Index']}")
    batch_size = int(row['batch_size'])
    max_epochs = 5
#     Create the command to start the job
    filename = f"{int(row['Index'])}-nodes{int(row['num_nodes'])}-batch{batch_size}-epochs{max_epochs}"
    command = f"/home/{SSH_USER}/bd/spark/bin/spark-submit "
    command += f"--master {masterip} --driver-cores 1 "
    command += f"--driver-memory 1G --total-executor-cores {int(row['total_executor_cores'])} --executor-cores {int(row['executor_cores'])} --executor-memory {row['memory_size']} "
    command += f"--py-files \"/home/{SSH_USER}/bd/spark/lib/bigdl-0.11.0-python-api.zip\" "
    command += f"--properties-file \"/home/{SSH_USER}/bd/spark/conf/spark-bigdl.conf\" "
    command += f"--jars \"/home/{SSH_USER}/bd/spark/lib/bigdl-SPARK_2.3-0.11.0-jar-with-dependencies.jar\" "
    command += f"--conf \"spark.driver.extraClassPath=/home/{SSH_USER}/bd/spark/lib/bigdl-SPARK_2.3-0.11.0-jar-with-dependencies.jar\" "
    command += f"--conf \"spark.executer.extraClassPath=bigdl-SPARK_2.3-0.11.0-jar-with-dependencies.jar,/home/{SSH_USER}/bd/codes/bi-rnn.py\" "
    command += f"/home/{SSH_USER}/bd/codes/{row['model']}.py "
    command += f"--action train --dataPath \"/tmp/mnist\" --batchSize {batch_size} --endTriggerNum {max_epochs} "
    command += f"--learningRate 0.01 --learningrateDecay 0.0002"
#     Split the command using a shell utility for the subprocess call
    splitted = shlex.split(command)
#     Create output files
    f_out = open(f"raw/{ex_id}/{ex_id}-{filename}.out", 'w')
    f_err = open(f"raw/{ex_id}/{ex_id}-{filename}.err", 'w')
    
#     Run the job
    process = subprocess.run(splitted,
                     stdout=f_out, 
                     stderr=f_err,
                     universal_newlines=True)